# Package Installation and import 📥
Here, we will install and import necessary packages for our code. 
These contain extra functions that we will need to make our program work

In [ ]:
import os
HOME = os.getcwd() # Get the current working directory
import ultralytics
from ultralytics import YOLO # Import YOLO class. This class is used to create a YOLOv8 model
from IPython.display import display, Image
from roboflow import Roboflow
import torch
from tqdm import tqdm

In [ ]:
HOME

_______________________________________________________________________________________________

# Loading the dataset 🌐
We will first create the place where we will store the dataset.
Then, we will import a pre-existing dataset from the web using *Roboflow*. You can later try to use a different dataset by going onto https://universe.roboflow.com/ and finding another dataset. Then, change the **datasetname** variable. Ensure that the dataset you select is labeled using the Oriented Bounding Box (OBB) format. Datasets not in OBB format will not work correctly with OBB-specific YOLO models like yolov8n-obb

In [ ]:
#create the directory for the datasets:
datasets_dir = os.path.join(HOME, "datasets_obb")
os.makedirs(datasets_dir, exist_ok=True)
os.chdir(datasets_dir)

# Download the dataset from Roboflow
datasetname = "yolo-obb-training-3"
rf = Roboflow(api_key="f5pOg5frjQ3XAImBbLxX")
project = rf.workspace("ryan-1ox6k").project("yolo-obb-training")
dataset = project.version(3).download("yolov8-obb")

# The Dataset 🌐
Get acquainted with your dataset. Let us try looking at one element from it:

In [ ]:
#get the first element from the dataset (code on computer)
sample = f"{HOME}/datasets_obb/{datasetname}/test/images/2T7OkT9RmTY_jpg.rf.7f68fbcbf3cd762eeefb2c42526c7807.jpg"

#get the full address of elements in the dataset
datasets_dir = os.path.join(HOME, "datasets_obb")
images = [os.path.join(datasets_dir, datasetname, "train", "images", image) for image in os.listdir(os.path.join(datasets_dir, datasetname, "train", "images"))]
labels = [os.path.join(datasets_dir, datasetname, "train", "labels", label) for label in os.listdir(os.path.join(datasets_dir, datasetname, "train", "labels"))]
#display five random pictures alongside their labels

for i in range(len(images)):
    display(Image(filename=images[i]))
    print(labels[i])

_______________________________________________________________________________________________

# Training the model 👨‍🏫
We will now simply feed the dataset to YOLO so that it learns how to recognise elements from it. 

In [ ]:
%cd {HOME}
output = f"{HOME}/runs/detect"
data_path = os.path.join(datasets_dir, datasetname, "data.yaml")
model = YOLO("yolov8n-obb.yaml")
results = model.train(data= data_path, epochs= 300, imgsz=640, plots=True, project= output)  # Set custom project directory

_______________________________________________________________________________________________

# Model Evaluation 📊
When we are analysing how well YOLO is at predicting the contents of an image, there are several metrics we can use.
The most important ones are the **training loss** and the **validation loss**. The lower these values are, the better your algorithm is at predicting data. 

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

# Furthermore, here is the F-1 Curve 📈
The F-1 curve tells us the overall performance of our model. It is particularly insightful because it **accounts for underrepresented classes**.
Imagine you have a thousand pictures of dogs and five of cats. You might have high accuracy if you always output dogs, but your F1 score will reflect this issue. 

In [ ]:
Image(filename=f'{HOME}/runs/detect/train/F1_curve.png', width=600)

_______________________________________________________________________________________________

## Testing the model ✍️
Previously, the model only saw pictures in the **train** folder. Now, we will show it the pictures in the **test** folder, pictures the model has never seen before. Based on how good the model's performance is with the test images, we can have an idea of what the model's performance with data in the real world will be.

## Here is the image we want to test 😽

In [ ]:
test1 =  f"{HOME}/datasets_obb/yolo-obb-training-3/test/images/photo_8_2025-05-16_16-54-24_jpg.rf.6a5e18134f5bd36b0cee933661939370.jpg"
Image(test1)

## Test our model 📝

In [ ]:
# Load a model
%cd {HOME}
model_path=f"{HOME}/runs/detect/train/weights/best.pt"

ourmodel = YOLO(model_path)

# Run batched inference on a list of images
results_2 = ourmodel(test1)  # list of 1 Results object

# Process results list
for result in results_2:
    result.show()  # display to screen

    # Print the Oriented Bounding Boxes (OBB) coordinates
    obbs = result.obb  # contains all obb
    for i, obb in enumerate(obbs):
        xywhr = obb.xywhr.cpu().numpy()[0]  # [x_center, y_center, width, height, rotation]
        print(f"OBB {i+1}: {xywhr}")
   